In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
import accelerate
import jsonlines
import torch
import torch.nn as nn
from torch.utils.checkpoint import checkpoint
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import os, csv, re
import openai
import inspect
import typing_extensions as typing


from modelsmith import Forge, VertexAIGenerativeModel

from typing import List
from pydantic import ValidationError, BaseModel, Field
import pandas as pd
from pathlib import Path
import jsonlines as jl
import json
import time
from tqdm import tqdm
 

import google.generativeai as genai
from google.generativeai.types import RequestOptions
from google.api_core import retry
from google.auth import default, transport
from modelsmith import Forge, VertexAIGenerativeModel
from vertexai.generative_models import GenerationConfig, GenerativeModel, Part
from dotenv import load_dotenv
import vertexai

load_dotenv('/Users/guida/llm_argument_tasks/.env')

api_key = os.environ.get('OPENAI_API_KEY')
PROJECT_ID = os.environ.get('GEMINI_PROJECT_ID')
LOCATION = "us-central1"

vertexai.init(
        project="leas-team",
    )

In [ ]:
class ArgumentSpan(typing.TypedDict):
    id: str 
    span: str 

In [ ]:
# Dictionary for label-to-argument mappings for each topic
topic_label_to_argument = {
    "abortion": {
        "p-right": "Abortion is a woman’s right.",
        "p-rape": "Rape victims need it to be legal.",
        "p-not_human": "A fetus is not a human yet, so it's okay to abort.",
        "p-mother_danger": "Abortion should be allowed when a mother's life is in danger.",
        "p-baby_ill_treatment": "Unwanted babies are ill-treated by parents and/or not always adopted.",
        "p-birth_ctrl": "Birth control fails at times and abortion is one way to deal with it.",
        "p-not_murder": "Abortion is not murder.",
        "p-sick_mom": "Mother is not healthy/financially solvent.",
        "p-other": "Others",
        "c-adopt": "Put baby up for adoption.",
        "c-kill": "Abortion kills a life.",
        "c-baby_right": "An unborn baby is a human and has the right to live.",
        "c-sex": "Be willing to have the baby if you have sex.",
        "c-bad_4_mom": "Abortion is harmful for women.",
        "c-other": "Others"
    },
    "gayRights": {
        "p-normal": "Gay marriage is like any other marriage.",
        "p-right_denied": "Gay people should have the same rights as straight people.",
        "p-no_threat_for_child": "Gay parents can adopt and ensure a happy life for a baby.",
        "p-born": "People are born gay.",
        "p-religion": "Religion should not be used against gay rights.",
        "p-Other": "Others",
        "c-religion": "Religion does not permit gay marriages.",
        "c-abnormal": "Gay marriages are not normal/against nature.",
        "c-threat_to_child": "Gay parents cannot raise kids properly.",
        "c-gay_problems": "Gay people have problems and create social issues.",
        "c-Other": "Others"
    },
    "obama": {
        "p-economy": "Fixed the economy.",
        "p-War": "Ending the wars.",
        "p-republicans": "Better than the republican candidates.",
        "p-decision_policies": "Makes good decisions/policies.",
        "p-quality": "Has qualities of a good leader.",
        "p-health": "Ensured better healthcare.",
        "p-foreign_policies": "Executed effective foreign policies.",
        "p-job": "Created more jobs.",
        "p-Other": "Others",
        "c-economy": "Destroyed our economy.",
        "c-War": "Wars are still on.",
        "c-job": "Unemployment rate is high.",
        "c-health": "Healthcare bill is a failure.",
        "c-decision_policies": "Poor decision-maker.",
        "c-republicans": "We have better republicans than Obama.",
        "c-quality": "Not eligible as a leader.",
        "c-foreign_policies": "Ineffective foreign policies.",
        "c-Other": "Others"
    },
    "marijuana": {
        "p-not_addictive": "Not addictive.",
        "p-medicine": "Used as a medicine for its positive effects.",
        "p-legal": "Legalized marijuana can be controlled and regulated by the government.",
        "p-right": "Prohibition violates human rights.",
        "p-no_damage": "Does not cause any damage to our bodies.",
        "p-Other": "Others",
        "c-health": "Damages our bodies.",
        "c-mind": "Responsible for brain damage.",
        "c-illegal": "If legalized, people will use marijuana and other drugs more.",
        "c-crime": "Causes crime.",
        "c-addiction": "Highly addictive.",
        "c-Other": "Others"
    }
}

In [ ]:
def classify_text(id: str, comment_text: str, topic: str, argument_text: str) -> dict:
    model = genai.GenerativeModel("gemini-1.5-flash")
    
    safety_settings = {
        "HARM_CATEGORY_HARASSMENT": "block_none",
        "HARM_CATEGORY_HATE_SPEECH": "block_none",
        "HARM_CATEGORY_SEXUALLY_EXPLICIT": "block_none",
        "HARM_CATEGORY_DANGEROUS_CONTENT": "block_none"
    }
    prompt = f"""
            Analyze the given comment in relation to a specific argument about {topic}. You need to:
            Identify the relevant span of text where the comment makes use of the given argument. 
            Provide the exact span of the text in the comment that makes use of the argument.
            Only report the exact original fragment of text. Do NOT paraphrase. Do NOT include any additional text.
            
            The argument to analyze is: {argument_text}
            
            Provide your response in the following JSON format:
            
            {{
                "id": "{id}",
                "span": "the relevant span of text"
            }}
            
            Analyze the following comment in relation to the given argument:
            {comment_text},
            """
    response = model.generate_content(
                    prompt,
                    generation_config=genai.types.GenerationConfig(
                        response_mime_type="application/json",
                        response_schema=ArgumentSpan,
                        temperature=0,
                        top_p=1,
                    ),
                    safety_settings=safety_settings
    )
                
    return response.text

In [ ]:
def process_dataframe_comments(df: pd.DataFrame, topic: str) -> List[dict]:
    label_to_argument = topic_label_to_argument.get(topic, {}) 
    with jsonlines.open(f'yru_{topic}_span_identification_gemini_original.jsonl', mode='a') as writer:
        for idx, row in tqdm(df.iterrows(), desc="Processing comments", unit="comment", total=len(df)):
            comment_id = row['id'] 
            comment_text = row['text']  
            comment_label = row['label']  

            argument_text = label_to_argument.get(comment_label)

            try:
                gemini = classify_text(
                    id=comment_id,
                    comment_text=comment_text,
                    topic=topic,
                    argument_text=argument_text
                )
                
                classification = json.loads(gemini)
                writer.write(classification)

            except json.JSONDecodeError as e:
                print(f"JSONDecodeError for comment: {comment_text[:50]}... - Error: {e}")
                error_entry = {"id": comment_id, "span": ""}
                writer.write(error_entry)
                continue

            except Exception as e:
                print(f"An unexpected error occurred for comment: {comment_text[:50]}... - Error: {e}")
                error_entry = {"id": comment_id, "span": ""}
                writer.write(error_entry)
                continue

In [ ]:
ab = pd.read_csv('/Users/guida/llm_argument_tasks/clean_data/yru_abortion.csv')
topic = 'abortion'

process_dataframe_comments(ab, topic)

Processing comments:  12%|█▏        | 92/739 [00:49<05:57,  1.81comment/s]

An unexpected error occurred for comment: "I have simply seen no argument that comes close t... - Error: Unknown field for Candidate: finish_message


Processing comments:  38%|███▊      | 278/739 [02:29<03:48,  2.02comment/s]

An unexpected error occurred for comment: 1. I've yet to see anyone ask a pro-lifer this que... - Error: Unknown field for Candidate: finish_message


Processing comments:  88%|████████▊ | 651/739 [05:53<00:44,  1.96comment/s]

An unexpected error occurred for comment: To be honest, yes it is in some way "murder," but ... - Error: Unknown field for Candidate: finish_message


Processing comments: 100%|██████████| 739/739 [06:42<00:00,  1.84comment/s]


In [ ]:
ma = pd.read_csv('/Users/guida/llm_argument_tasks/clean_data/yru_marijuana.csv')
topic = 'marijuana'

process_dataframe_comments(ma, topic)

Processing comments:   0%|          | 0/691 [00:00<?, ?comment/s]

Processing comments: 100%|██████████| 691/691 [06:07<00:00,  1.88comment/s]


In [ ]:
oba = pd.read_csv('/Users/guida/llm_argument_tasks/clean_data/yru_obama.csv')
topic = 'obama'

process_dataframe_comments(oba, topic)

Processing comments:   0%|          | 0/646 [00:00<?, ?comment/s]

Processing comments:  59%|█████▉    | 380/646 [03:16<02:15,  1.96comment/s]

An unexpected error occurred for comment: obama only won because of all the welfare niggas o... - Error: Unknown field for Candidate: finish_message


Processing comments: 100%|█████████▉| 645/646 [05:41<00:00,  1.67comment/s]

An unexpected error occurred for comment: Iâm a Dr Ron Paul kind of guy. I donât vote ba... - Error: Unknown field for Candidate: finish_message


Processing comments: 100%|██████████| 646/646 [05:42<00:00,  1.89comment/s]

An unexpected error occurred for comment: Iâm a Dr Ron Paul kind of guy. I donât vote ba... - Error: Unknown field for Candidate: finish_message


In [ ]:
gm = pd.read_csv('/Users/guida/llm_argument_tasks/clean_data/yru_gayrights.csv')
topic = 'gayRights'

process_dataframe_comments(gm, topic)

Processing comments:   0%|          | 0/772 [00:00<?, ?comment/s]

Processing comments: 100%|██████████| 772/772 [06:57<00:00,  1.85comment/s]
